# Estratégias para Previsão de Múltiplos Horizontes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import train_test_split
from preprocessing import make_shifts
from metrics import all_metrics_from_dataframes, mse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from direct import Direct
from multiout import Multiout
from recursive import Recursive
from arima import Arima

## 1. Pré-processamento

In [2]:
df = pd.read_csv('../data/australia_wind_power.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])                             # 0, 0, 34 NaN
df = df.set_index('timestamp').rolling(6).mean().iloc[6::6]
df = df[['BOCORWF1', 'MACARTH1','STARHLWF']].interpolate(method='polynomial', order=5, axis=0).clip(lower=0)   

In [3]:
# for label, serie in df.items():
#     plt.figure(figsize=(15,4))
#     plt.title(label)
#     plt.plot(serie)
#     plt.savefig(f'../images/{label}.png')

In [4]:
# for label, serie in df.items():
#     plot_pacf(serie, title=f'PACF {label}', lags=40)
#     plt.savefig(f'../images/PACF_{label}.png')

In [5]:
# for label, serie in df.items():
#     plot_acf(serie, title=f'ACF {label}', lags=100)
#     plt.savefig(f'../images/ACF_{label}.png')

In [6]:
LAGS = 5
HORIZONS = 3
CENTRAL = 'MACARTH1'
INITS = 1 
HIDDEN_LAYERS = [10,5]
ACTIVATION = 'relu'
OPTIMIZER = 'adam'
LOSS = 'mse'
VALIDATION_SPLIT = 0.2
EPOCHS = 200

serie = df[CENTRAL]
train, test = train_test_split(serie, test_size=.33, shuffle=False)
X_train, Y_train = make_shifts(train, LAGS, HORIZONS, dropnan=True, savename=CENTRAL+'_train')
X_test, Y_test = make_shifts(test, LAGS, HORIZONS, dropnan=False, savename=CENTRAL+'_test')

def result(strategy, type):
        model = strategy(type, n_horizons=HORIZONS, n_inits=INITS)
        model.build(hidden_layers=HIDDEN_LAYERS, input_size=LAGS, activation=ACTIVATION, optimizer=OPTIMIZER, loss=LOSS)
        model.train(X_train, Y_train, validation_split=VALIDATION_SPLIT, epochs=EPOCHS)
        return model.predict(X_test)

# 2. Estratégias
## 2.1 Direta

In [9]:
D_MLP = result(Direct, 'MLP')
all_metrics_from_dataframes(D_MLP, Y_test)

Epoch 1/200
294/294 [==============================] - 0s 2ms/step - loss: 23951.4414 - val_loss: 37200.1211
Epoch 2/200
294/294 [==============================] - 0s 1ms/step - loss: 14949.8154 - val_loss: 23401.0449
Epoch 3/200
294/294 [==============================] - 0s 1ms/step - loss: 9615.1504 - val_loss: 14025.1553
Epoch 4/200
294/294 [==============================] - 0s 1ms/step - loss: 5405.3428 - val_loss: 5895.5771
Epoch 5/200
294/294 [==============================] - 0s 1ms/step - loss: 2373.5850 - val_loss: 2230.6418
Epoch 6/200
294/294 [==============================] - 0s 1ms/step - loss: 1682.5458 - val_loss: 2170.7566
Epoch 7/200
294/294 [==============================] - 0s 995us/step - loss: 1630.0775 - val_loss: 2040.2214
Epoch 8/200
294/294 [==============================] - 0s 1ms/step - loss: 1609.9550 - val_loss: 1972.9275
Epoch 9/200
294/294 [==============================] - 0s 1ms/step - loss: 1594.7806 - val_loss: 1964.1031
Epoch 10/200
294/294 [========

,t+1,t+2,t+3
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,110.290764,112.313858,111.396248
...,...,...,...
5777,29.929808,30.377329,29.759266
5778,21.914230,22.348413,21.782522
5779,16.369675,16.899168,16.385571
5780,10.466894,11.344240,10.924514


In [ ]:
D_LSTM = result(Direct, 'LSTM')
all_metrics_from_dataframes(D_LSTM, Y_test)

### 2.2 Múltiplas saídas

In [ ]:
M_MLP = result(Multiout, 'MLP')
all_metrics_from_dataframes(M_MLP, Y_test)

In [ ]:
M_LSTM = result(Multiout, 'LSTM')
all_metrics_from_dataframes(M_LSTM, Y_test)

### 2.3 Recursiva

In [ ]:
R_MLP = result(Recursive, 'MLP')
all_metrics_from_dataframes(R_MLP, Y_test)

In [ ]:
R_LSTM = result(Recursive, 'LSTM')
all_metrics_from_dataframes(R_LSTM, Y_test)

In [ ]:
a = slice(200,300)
for h in range(1,HORIZONS+1):
  #  plt.plot(predD_MLP[f't+{h}'].values[a], label='d-mlp')
    plt.plot(predD_LSTM[f't+{h}'].values[a], label='d-lstm')
  #  plt.plot(predM_MLP[f't+{h}'].values[a], label='m-mlp')
    plt.plot(predM_LSTM[f't+{h}'].values[a], label='m-lstm')
  #  plt.plot(predR_MLP[f't+{h}'].values[a], label='r-mlp')
    plt.plot(predR_LSTM[f't+{h}'].values[a], label='r-lstm')
    plt.plot(y_test[f't+{h}'].values[a], label='obs')
    plt.legend()
    plt.title(f'Horizon {h}')
    plt.show()

# 3. Modelos de Referência

## 3.1 ARIMA

In [12]:
M_ARIMA = Arima(n_horizons=HORIZONS, order=(9,1,0))
predM_ARIMA = M_ARIMA.predict_multiout(train.values, test.values)
predM_ARIMA

KeyboardInterrupt: 

In [ ]:
R_ARIMA = Arima(n_horizons=HORIZONS, order=(9,1,0))
predR_ARIMA = R_ARIMA.predict_recursive(train, test)
predR_ARIMA

In [ ]:
plt.plot(test, label='test')
plt.plot(pred_list, label='pred')
plt.legend()
plt.show()

## 3.2 Persistência